# BIG Data analysis and BI visualsing the data

# Installing and Importing the Required Libraries

In [1]:
#!pip install mlxtend
import pandas as pd
import numpy as np

# Importing the data as df

Download dataset from : https://www.kaggle.com/carrie1/ecommerce-data?select=data.csv

In [2]:
#importing the data
df = pd.read_csv('events.csv',encoding= 'utf-8')
df.shape

(885129, 9)

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [4]:
df.isnull().sum().sort_values(ascending=False)

category_code    236219
brand            212364
user_session        165
event_time            0
event_type            0
category_id           0
product_id            0
price                 0
user_id               0
dtype: int64

# Cleaning the data

In [5]:
df1 = df.dropna()

In [6]:
df1.describe()

# Here we can see quantity has some negative values which is a part of incorrect data so we will drop such entries

,product_id,category_id,price,user_id
count,5.162560e+05,5.162560e+05,516256.000000,5.162560e+05
mean,2.165876e+06,2.144416e+18,188.908173,1.515916e+18
std,1.497812e+06,2.019302e+14,347.516805,3.565386e+07
min,8.540000e+02,2.144416e+18,0.900000,1.515916e+18
25%,8.554510e+05,2.144416e+18,45.240000,1.515916e+18
50%,1.682541e+06,2.144416e+18,105.870000,1.515916e+18
75%,3.828320e+06,2.144416e+18,249.380000,1.515916e+18
max,4.183866e+06,2.227847e+18,64771.060000,1.515916e+18


In [9]:
#selecting data where quantity > 0
df1 = df1[df1.price > 0]

In [10]:
df1.describe()

,product_id,category_id,price,user_id
count,5.162560e+05,5.162560e+05,516256.000000,5.162560e+05
mean,2.165876e+06,2.144416e+18,188.908173,1.515916e+18
std,1.497812e+06,2.019302e+14,347.516805,3.565386e+07
min,8.540000e+02,2.144416e+18,0.900000,1.515916e+18
25%,8.554510e+05,2.144416e+18,45.240000,1.515916e+18
50%,1.682541e+06,2.144416e+18,105.870000,1.515916e+18
75%,3.828320e+06,2.144416e+18,249.380000,1.515916e+18
max,4.183866e+06,2.227847e+18,64771.060000,1.515916e+18


# Insights from the dataset

## Customers Insights

This segment will answer the questions like :
1. Who all are my loyal customers?
2. Which customers have ordered most frequently?
3. Which customers contribute the most to my revenue?

In [12]:
import datetime

### save data to duckdb and use this saved data for subsequent uses

In [17]:
import duckdb
conn = duckdb.connect('event.db')

# Register the DataFrame as a DuckDB table
conn.register('data1', df1)

# Create a new table from the registered DataFrame
conn.execute('CREATE TABLE event_data AS SELECT * FROM df1')

# Fetch data back into a DataFrame
data1 = conn.execute("SELECT * FROM event_data").fetchdf()
df1 = data1
df.to_csv("event_filtered.csv", index=False,  encoding='utf-8')

This segment will display how FREE items impact the No. of Orders. This answers how will the discounts and offers impact my sale?

In [14]:
df1.price.describe()
# With minimum Unit Price = 0 there can be either some incorrect entry or free items

count    516256.000000
mean        188.908173
std         347.516805
min           0.900000
25%          45.240000
50%         105.870000
75%         249.380000
max       64771.060000
Name: price, dtype: float64

In [25]:
# Creating a new df of free items
free_items = df1[df1['UnitPrice'] == 0]
free_items.head()

,InvoiceNo,StockCode,year_month,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
9302,537197,22841,201012,12,7,14,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany,0.0
33576,539263,22580,201012,12,4,14,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom,0.0
40089,539722,22423,201012,12,2,13,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE,0.0
47068,540372,22090,201101,1,4,16,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom,0.0
47070,540372,22553,201101,1,4,16,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom,0.0
